In [1]:
import pandas as pd
import mysql.connector
query = "SELECT * FROM market"
mydb = mysql.connector.connect(
      host="172.31.50.91",
      user="guest",
      password="MH#123456",
       database="astocks"
    )
df = pd.read_sql(query, mydb)

C:\Users\19550\AppData\Local\Temp\ipykernel_15404\909013756.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, mydb)


In [2]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np

from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import os
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from numpy import newaxis

In [ ]:
min_max = MinMaxScaler(feature_range=(0, 1))
train_scaled = min_max.fit_transform(training_data)

In [ ]:
# Define the number of time steps
time_steps = 3

# Loop through the data to create partitions
for i in range(time_steps, train_scaled.shape[0]):
    # Create a partition of the previous 3 days' data
    x_train.append(train_scaled[i - time_steps:i])

    # Append the next day's Close price to the label array
    y_train.append(train_scaled[i, 0])

In [ ]:
def create_long_short_term_memory_model(x_train):
    model = Sequential()
    # 1st layer with Dropout regularisation
    # * units = add 100 neurons is the dimensionality of the output space
    # * return_sequences = True to stack LSTM layers so the next LSTM layer has a three-dimensional sequence input
    # * input_shape => Shape of the training dataset
    model.add(LSTM(units=100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    # 20% of the layers will be dropped
    model.add(Dropout(0.2))
    # 2nd LSTM layer
    # * units = add 50 neurons is the dimensionality of the output space
    # * return_sequences = True to stack LSTM layers so the next LSTM layer has a three-dimensional sequence input
    model.add(LSTM(units=50, return_sequences=True))
    # 20% of the layers will be dropped
    model.add(Dropout(0.2))
    # 3rd LSTM layer
    # * units = add 50 neurons is the dimensionality of the output space
    # * return_sequences = True to stack LSTM layers so the next LSTM layer has a three-dimensional sequence input
    model.add(LSTM(units=50, return_sequences=True))
    # 50% of the layers will be dropped
    model.add(Dropout(0.5))
    # 4th LSTM layer
    # * units = add 50 neurons is the dimensionality of the output space
    model.add(LSTM(units=50))
    # 50% of the layers will be dropped
    model.add(Dropout(0.5))
    # Dense layer that specifies an output of one unit
    model.add(Dense(units=1))
    model.summary()
    tf.keras.utils.plot_model(model, to_file=os.path.join(project_folder, 'model_lstm.png'), show_shapes=True,
                              show_layer_names=True)
    return model

In [ ]:
defined_metrics = [
    tf.keras.metrics.MeanSquaredError(name='MSE')
]

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=defined_metrics)
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test),
                    callbacks=[callback])

In [ ]:
x_test = []
y_test = []
for i in range(time_steps, test_scaled.shape[0]):
    x_test.append(test_scaled[i - time_steps:i])
    y_test.append(test_scaled[i, 0])

x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [146]:
code = "000777.SZ"


In [165]:
data = df[df['codenum'] == code]
data = data.astype({'td':'int'})

In [166]:
data = data.reset_index()


In [167]:
data = data.drop(columns=['id','codenum'])

In [168]:
cut = int(len(data)*0.80)

In [169]:
cut

5045

In [170]:
data = data['close']

In [171]:


train_data = data.iloc[:cut]
test_data = data.iloc[cut:]


    

In [174]:
train_data

0        9.06
1        9.97
2       10.97
3       11.53
4       10.98
        ...  
5040    13.40
5041    13.39
5042    13.33
5043    13.14
5044    13.24
Name: close, Length: 5045, dtype: float64

In [175]:
data

0        9.06
1        9.97
2       10.97
3       11.53
4       10.98
        ...  
6302    13.38
6303    13.43
6304    13.59
6305    13.47
6306    13.60
Name: close, Length: 6307, dtype: float64

In [179]:

def normalise_zero_base(continuous):
    return continuous / continuous.iloc[0] - 1

def normalise_min_max(continuous):
    return (continuous - continuous.min()) / (data.max() - continuous.min())
def extract_window_data(continuous, window_len=5, zero_base=True):
    window_data = []
    for idx in range(len(continuous) - window_len):
        tmp = continuous[idx: (idx + window_len)].copy()
        if zero_base:
            tmp = normalise_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)
def prepare_data(continuous,  window_len=10, zero_base=True, test_size=0.2):
    X_train = extract_window_data(train_data, window_len, zero_base)
    X_test = extract_window_data(test_data, window_len, zero_base)
    y_train = train_data[window_len:].values
    y_test = test_data[window_len:].values
    if zero_base:
        y_train = y_train / train_data[:-window_len].values - 1
        y_test = y_test / test_data[:-window_len].values - 1

    return train_data, test_data, X_train, X_test, y_train, y_test
import numpy as np
     

In [177]:

from tensorflow.keras import layers

In [189]:
def build_lstm_model(input_data, output_size, neurons, activ_func='linear',dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(input_data.shape[1], input_data.shape[0])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    model.compile(loss=loss, optimizer=optimizer)
    return model
np.random.seed(245)
window_len = 5
test_size = 0.2
zero_base = True
lstm_neurons = 50
epochs = 20
batch_size = 32
loss = 'mse'
dropout = 0.24
optimizer = 'adam'
train_data, test_data, X_train, X_test, y_train, y_test = prepare_data(data,  window_len=window_len, zero_base=zero_base, test_size=test_size)
     

C:\Users\19550\AppData\Local\Temp\ipykernel_15404\3541479114.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  tmp = continuous[idx: (idx + window_len)].copy()
C:\Users\19550\AppData\Local\Temp\ipykernel_15404\3541479114.py:18: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test = test_data[window_len:].values
C:\Users\19550\AppData\Local\Temp\ipykernel_15404\3541479114.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future v

In [190]:
print(X_train.shape)

(5040, 5)


In [191]:
model = build_lstm_model(
    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
    optimizer=optimizer)
modelfit = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

Epoch 1/20


ValueError: in user code:

    File "C:\Users\19550\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\19550\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\19550\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\19550\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\19550\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\1955